In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('../output/categorized_test_statement_section.csv')
test_df

,statement,section,label,InferenceType,SectionType,Type
0,Women suffering from both claustrophobia and I...,"[""Inclusion Criteria:"", ""Women are eligible to...",Contradiction,NLI,Eligibility,Comparison
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", ""Women 18 years"", ""His...",Entailment,NLI,Eligibility,Single
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", ""Patients presenting f...",Entailment,NLI,Eligibility,Comparison
3,Patients currently taking part in the secondar...,"[""Inclusion Criteria:"", ""Be a female of any ra...",Entailment,NLI,Eligibility,Comparison
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1:"", ""Trastuzumab"", ""Participan...",Contradiction,Numerical,Intervention,Comparison
...,...,...,...,...,...,...
495,alcoholic patients are excluded from the prima...,"[""Inclusion Criteria:"", ""All participants will...",Contradiction,Numerical,Eligibility,Comparison
496,Patients with a histologically/cytologically c...,"[""Inclusion Criteria:"", ""Voluntarily signed an...",Entailment,NLI,Eligibility,Single
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1:"", ""Pregabalin150"", ""Patients...",Entailment,Numerical,Intervention,Comparison
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement:"", ""Ki67 Expression"", ""K...",Contradiction,Numerical,Results,Single


In [3]:
hypothesis_lst=test_df['statement'].values.tolist()
len(hypothesis_lst)

500

In [4]:
evidence_lst=test_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

500

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

500

In [6]:
import json
with open('../output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in test_df[['statement','section']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
scores

[0.10540307911788183,
 0.05143603071878439,
 0.0567025745568231,
 0.08978377811313051,
 0.12169340725126061,
 0.0,
 0.16582749190468873,
 0.1276865175151018,
 0.0,
 0.22611814131660077,
 0.12407683831813195,
 0.04281831075085723,
 0.06146126974425214,
 0.4730209831350068,
 0.054624251519553024,
 0.14805011919425942,
 0.3814545184568042,
 0.26447359362655465,
 0.008813545218841383,
 0.021800948308208598,
 0.04750121714849074,
 0.17850231395772118,
 0.06039382968010121,
 0.08338795650750498,
 0.24789912978337342,
 0.09065458736137637,
 0.0,
 0.08658333568355238,
 0.10847040328649,
 0.12036480806974431,
 0.14340415892866196,
 0.022620975465106382,
 0.35750069067345436,
 0.18057917122999106,
 0.1321619770718224,
 0.06153377905611968,
 0.10756662913196817,
 0.2721825475453859,
 0.10174268242173652,
 0.025381466445972032,
 0.321961649693751,
 0.14864427545529257,
 0.07545176690926522,
 0.08888814717092913,
 0.09073918205246806,
 0.16188375459426427,
 0.0,
 0.18341122369781215,
 0.14164853155

In [9]:
df2=test_df.copy()
df2['score']=scores
df2

,statement,section,label,InferenceType,SectionType,Type,score
0,Women suffering from both claustrophobia and I...,"[""Inclusion Criteria:"", ""Women are eligible to...",Contradiction,NLI,Eligibility,Comparison,0.105403
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", ""Women 18 years"", ""His...",Entailment,NLI,Eligibility,Single,0.051436
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", ""Patients presenting f...",Entailment,NLI,Eligibility,Comparison,0.056703
3,Patients currently taking part in the secondar...,"[""Inclusion Criteria:"", ""Be a female of any ra...",Entailment,NLI,Eligibility,Comparison,0.089784
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1:"", ""Trastuzumab"", ""Participan...",Contradiction,Numerical,Intervention,Comparison,0.121693
...,...,...,...,...,...,...,...
495,alcoholic patients are excluded from the prima...,"[""Inclusion Criteria:"", ""All participants will...",Contradiction,Numerical,Eligibility,Comparison,0.054109
496,Patients with a histologically/cytologically c...,"[""Inclusion Criteria:"", ""Voluntarily signed an...",Entailment,NLI,Eligibility,Single,0.128692
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1:"", ""Pregabalin150"", ""Patients...",Entailment,Numerical,Intervention,Comparison,0.192566
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement:"", ""Ki67 Expression"", ""K...",Contradiction,Numerical,Results,Single,0.315865


In [10]:
df2.to_csv('../output/categorized_bm25_score.csv',index=False)